# Debugging Your Homework

This notebook helps you interactively debug homework 5.

In [ ]:
# Enable autoreload to make debugging easier

%load_ext autoreload
%autoreload 2

In [ ]:
# Make sure the homework is installed as a package in the "editing" mode:

! pip install -e ..

In [ ]:
# Then, load all the necessary packages
# Note that there's warning from Wswiglal-redir-stdio that we will filter out

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")

from phys305_hw5 import *
import numpy as np
import matplotlib.pyplot as plt

from bilby.core.prior import Uniform, PowerLaw

## Getting the data: GW150914

Just like the tutorial, we'll analyse GW150914 in this homework.
Our first task is to use `Interferometer()` to load up some data.

In [ ]:
time_of_event = 1126259462.4

L1 = Interferometer('L1', time_of_event)
H1 = Interferometer('H1', time_of_event)

If your `Interferometer()` is implemented correctly, the following visualizations should work.

In [ ]:
fig, (ax0, ax1) = plt.subplots(2,1)

idxs = H1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we'll use in the analysis
ax0.loglog(H1.strain_data.frequency_array[idxs],
          np.abs(H1.strain_data.frequency_domain_strain[idxs]))
ax0.loglog(H1.power_spectral_density.frequency_array[idxs],
          H1.power_spectral_density.asd_array[idxs])

idxs = L1.strain_data.frequency_mask  # This is a boolean mask of the frequencies which we'll use in the analysis
ax1.loglog(L1.strain_data.frequency_array[idxs],
          np.abs(L1.strain_data.frequency_domain_strain[idxs]))
ax1.loglog(L1.power_spectral_density.frequency_array[idxs],
          L1.power_spectral_density.asd_array[idxs])

ax1.set_xlabel("Frequency [Hz]")
ax1.set_ylabel("Strain [strain/$\sqrt{Hz}$]")
plt.show()

We combine the two interferometers into a list for later use:

In [ ]:
interferometers = [H1, L1]

## Create a Prior

Here, we create a prior fixing everything except the chirp mass, mass ratio, phase, geocent_time, and luminosity distance parameters.
We narrow the domain of `phase`, `geocent_time`, and `luminosity_distance` based on the fitted values from the tutorial.
This allows us to simplify this homework.

In [ ]:
prior = bilby.core.prior.PriorDict()

prior['chirp_mass']   = Uniform(name='chirp_mass',   minimum=28.0,maximum=32)
prior['mass_ratio']   = Uniform(name='mass_ratio',   minimum=0.5, maximum=1)
prior['phase']        = Uniform(name="phase",        minimum=4.54-0.2,   maximum=4.54+0.2)
prior['geocent_time'] = Uniform(name="geocent_time", minimum=time_of_event-0.01, maximum=time_of_event+0.01)
prior['a_1']          =  0.0
prior['a_2']          =  0.0
prior['tilt_1']       =  0.0
prior['tilt_2']       =  0.0
prior['phi_12']       =  0.0
prior['phi_jl']       =  0.0
prior['dec']          =  -1.2232
prior['ra']           =  2.19432
prior['theta_jn']     =  1.89694
prior['psi']          =  0.532268
prior['luminosity_distance'] = PowerLaw(alpha=2, name='luminosity_distance', minimum=277, maximum=320, unit='Mpc', latex_label='$d_L$')

The above values will be useful for homework assignment `a5.py`.

We can now use `prior` to sample and compute a probability.

In [ ]:
sample = prior.sample()
print(sample)
print(prior.ln_prob(sample))

## Waveform Generator

Next, we use `WaveformGenerator(duration, sampling_frequency, start_time)` to create a waveform generator.
Note that, because the parameters `duration`, `sampling_frequency`, `start_time` are the same for both interferometers, we only need to create one waveform generator.

In [ ]:
print(
    L1.duration           == H1.duration,
    L1.sampling_frequency == H1.sampling_frequency,
    L1.start_time         == H1.start_time,
)

In [ ]:
waveform_generator = WaveformGenerator(L1.duration, L1.sampling_frequency, L1.start_time)

If your `WaveformGenerator()` is implemented correctly, the following plot should work.

In [ ]:
waveforms = waveform_generator.frequency_domain_strain(sample)

plt.loglog(waveform_generator.frequency_array, abs(waveforms['plus']))
plt.loglog(waveform_generator.frequency_array, abs(waveforms['cross']))

One may update the parameters in `sample` to generate a different waveform.

In [ ]:
sample['chirp_mass'] = 10
waveforms = waveform_generator.frequency_domain_strain(sample)

plt.loglog(waveform_generator.frequency_array, abs(waveforms['plus']))
plt.loglog(waveform_generator.frequency_array, abs(waveforms['cross']))

## Create a likelihood

For Bayesian inference, we need to evaluate the likelihood.

In [ ]:
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers, waveform_generator, priors=prior,
    time_marginalization=True, phase_marginalization=True, distance_marginalization=True)

We can now compute `log_likelihood()` for different `parameters`.

In [ ]:
for _ in range(10):
    likelihood.parameters = prior.sample()
    print(likelihood.log_likelihood())

## Run the analysis

Now that the prior is set-up and the likelihood is set-up (with the data and the signal mode), we can run our own MCMC sampler in `a3.py` to get the posterior result.

In [ ]:
samples = mcmc_sampler(likelihood, prior, [0.1, 0.01])

## Looking at the outputs

We can now use `a4.py` to print out statistics information and create a corner plot. 

In [ ]:
stat(samples, 'corner.png')